## Experiment #6

This experiment studies whether BLAH BLAH BLAH is superior to BLAH BLAH BLAH. 

In [ ]:
VERSION="v6"

In [1]:
import warnings
warnings.filterwarnings('ignore')
import regex as re
import os
import shutil
import sys
import importlib
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import celloracle as co
from scipy.stats import spearmanr as spearmanr

#      visualization settings
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.rcParams['figure.figsize'] = [6, 4.5]
plt.rcParams["savefig.dpi"] = 300

# self and output location
os.chdir("/home/ekernf01/Desktop/jhu/research/projects/perturbation_prediction/cell_type_knowledge_transfer/minimal_example/scripts")
try:
    os.makedirs("../results/" + VERSION)
except:
    pass

### Networks setup

This experiment uses the two default networks: the dense network and the CO default network. 

In [ ]:
networks = {"dense":"bleerrrrrrrghhhh"}

### Data setup

We use the Nakatake et al data. This experiment is on per-cluster versus shared regression models, so we run Leiden clustering at many different resolutions.

In [ ]:
ko_lab_esc_data = sc.read_h5ad("???")
for r in [i/10.0 for i in range(1,11,2)]:
    sc.tl.leiden(ko_lab_esc_data, resolution=r)
    new_name = "leiden_r="+str(round(r, 1))
    clusterResolutions.append(new_name)
    ko_lab_esc_data.obs[new_name] = ko_lab_esc_data.obs["leiden"]
    sc.pl.umap(ko_lab_esc_data, color = "leiden")
    ko_lab_esc_data.obs[new_name] = ko_lab_esc_data.obs[new_name].astype("category")
    ko_lab_esc_data.uns[new_name + "_colors"] = ko_lab_esc_data.uns["leiden_colors"]

### Training and test split

A few factors complicate the training-test split. 

- Perturbed genes may be absent from most base GRN's due to lack of motif information or ChIP data. These are excluded from the test data to avoid obvious failure cases.
- Perturbed genes may not be measured. These are excluded from test data because we don't know to what extent they were overexpressed.

In both cases, we still use those perturbed profiles as training data, hoping they will provide useful info about attainable cell states and downstream causal effects. There are so many factors ineligible for use as test data that we use all the eligible ones for test and the only ineligible ones for training. 

In [12]:
test_set_regulators = set.intersection(*[set(networks[key]['gene_short_name']) 
                                      for key in networks if not key == 'empty'])
test_set_regulators = test_set_regulators.difference(perturbed_but_not_measured_genes)
testSetPerturbations     = set(ko_lab_esc_data.obs['perturbation']).intersection(test_set_regulators)
trainingSetPerturbations = set(ko_lab_esc_data.obs['perturbation']).difference(test_set_regulators)
ko_lab_esc_data_heldout  = ko_lab_esc_data[ko_lab_esc_data.obs["perturbation"].isin(testSetPerturbations),    :]
ko_lab_esc_data_train    = ko_lab_esc_data[ko_lab_esc_data.obs["perturbation"].isin(trainingSetPerturbations),:]
ko_lab_esc_data_train.obs['perturbation'].unique()
perturbationsToPredict = [(gene, ko_lab_esc_data_heldout[sample, gene].X[0,0]) for sample,gene in enumerate(ko_lab_esc_data_heldout.obs['perturbation'])] 
print("Example perturbations formatted as \n (gene, expression after perturbation)")
print(perturbationsToPredict[0:5])
print("Test set size:")
print(len(testSetPerturbations))
print("Training set size:")
print(len(trainingSetPerturbations))
# ko_lab_esc_data_train


Example perturbations formatted as 
 (gene, expression after perturbation)
[('AATF', 3.8030212), ('AATF', 3.3206005), ('ALX3', 2.829177), ('ALX3', 3.583377), ('ALX4', 1.1726332)]
Test set size:
500
Training set size:
13


### OK, let's do this. 

In [13]:
n_networks = len(networks.keys())
n_res = len(clusterResolutions)
experiments = pd.DataFrame({"network":[n for n in networks.keys()]*n_res, 
                            "cluster_resolution":clusterResolutions + clusterResolutions,
                            "p":[1]*n_networks*n_res,
                            "threshold_number":[int(network_sizes['numEdges'].max())]*n_networks*n_res,
                            "pruning":["none"]*n_networks*n_res})
experiments["index"] = experiments.index
experiments.to_csv("../results/" + VERSION + "/networkExperiments.csv")

In [ ]:
predictions = {
    i: trainCausalModelAndPredict(expression=ko_lab_esc_data_train,
                                  baseNetwork=networks[experiments.loc[i,'network']],
                                  memoizationName="../results/" + VERSION + "/" + str(i) + ".celloracle.oracle", 
                                  perturbations=perturbationsToPredict,
                                  clusterColumnName = experiments.loc[i, "cluster_resolution"],
                                  pruningParameters = {"p":experiments.loc[i,'p'], 
                                                       "threshold_number":experiments.loc[i,'threshold_number']}) 
    for i in experiments.index
}


Working on ../results/v6/0.celloracle.oracle


/home/ekernf01/.local/share/r-miniconda/envs/cell_type_grn_transfer/lib/python3.9/site-packages/celloracle/trajectory/neighbors.py:402: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  if type(mknn) is not sparse.csr.csr_matrix:


  0%|          | 0/1 [00:00<?, ?it/s]

inferring GRN for 0...


  0%|          | 0/2407 [00:00<?, ?it/s]

...

  0%|          | 0/2603 [00:00<?, ?it/s]

genes_in_gem: 2603
models made for 2407 genes


/home/ekernf01/.local/share/r-miniconda/envs/cell_type_grn_transfer/lib/python3.9/site-packages/celloracle/utility/hdf5_processing.py:75: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if attribute.dtype is not np.dtype(np.object):
/home/ekernf01/.local/share/r-miniconda/envs/cell_type_grn_transfer/lib/python3.9/site-packages/celloracle/utility/hdf5_processing.py:75: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if attribute.dtype is not np.dtype(np.object):
/home/ekernf01/.

Predicting AATF
Predicting AATF
Predicting ALX3
Predicting ALX3
Predicting ALX4
Predicting ALX4
Predicting ANKRD22
Prediction failed for ANKRD22 with error Gene ANKRD22 is not included in the base GRN; It is not TF or TF motif information is not available. Cannot perform simulation.
Predicting ANKRD22
Prediction failed for ANKRD22 with error Gene ANKRD22 is not included in the base GRN; It is not TF or TF motif information is not available. Cannot perform simulation.
Predicting AR
Predicting AR
Predicting ARID3A
Predicting ARID3A
Predicting ARNT2
Predicting ARNT2
Predicting ASCL1
Predicting ASCL1
Predicting ASCL2
Predicting ASCL2
Predicting ASH2L
Prediction failed for ASH2L with error Gene ASH2L is not included in the base GRN; It is not TF or TF motif information is not available. Cannot perform simulation.
Predicting ASH2L
Prediction failed for ASH2L with error Gene ASH2L is not included in the base GRN; It is not TF or TF motif information is not available. Cannot perform simulation

Predicting GATA3
Predicting GATA3
Predicting GATA6
Predicting GATA6
Predicting GBX2
Predicting GBX2
Predicting GLI1
Predicting GLI1
Predicting GLIS2
Predicting GLIS2
Predicting GRHL2
Predicting GRHL2
Predicting GSC
Predicting GSC
Predicting GTF2B
Predicting GTF2B
Predicting GTF2F1
Predicting GTF2F1
Predicting GTF3C2
Predicting GTF3C2
Predicting H2AFZ
Prediction failed for H2AFZ with error Gene H2AFZ is not included in the base GRN; It is not TF or TF motif information is not available. Cannot perform simulation.
Predicting H2AFZ
Prediction failed for H2AFZ with error Gene H2AFZ is not included in the base GRN; It is not TF or TF motif information is not available. Cannot perform simulation.
Predicting HDAC2
Predicting HDAC2
Predicting HDAC3
Predicting HDAC3
Predicting HDAC6
Predicting HDAC6
Predicting HDAC8
Predicting HDAC8
Predicting HES1
Predicting HES1
Predicting HESX1
Predicting HESX1
Predicting HEY1
Predicting HEY1
Predicting HHEX
Predicting HHEX
Predicting HIST2H3C
Prediction fai

In [ ]:
predictions[0]

### Evaluation

We compute the correlation of the predictions with held-out perturbations.

In [ ]:
controlIndex = ko_lab_esc_data_train.obs["perturbation"]=="Control"
evaluationResults = {}
for i in predictions:
    evaluationResults[i] = \
        evaluateCausalModel(ko_lab_esc_data_heldout, 
                            predictions[i],   
                            baseline = ko_lab_esc_data_train[controlIndex,:].X.mean(axis=0),     
                            doPlots=False)[0]
    evaluationResults[i]["index"] = i
evaluationResults = pd.concat(evaluationResults)
evaluationResults = evaluationResults.merge(experiments, how = "left")
evaluationResults = pd.DataFrame(evaluationResults.to_dict())
evaluationResults.head()

In [ ]:
noPredictionMade = evaluationResults.iloc[[x==0 for x in evaluationResults["spearman"]],:]['perturbation']
noPredictionMade = set(noPredictionMade)
noPredictionMade
evaluationResults["somePredictionRefused"] = evaluationResults["perturbation"].isin(noPredictionMade) 
evaluationResults.to_csv("../results/"+ VERSION +"/networksExperimentEvaluation.csv")
evaluationResults.head()

In [ ]:
baseNetworkComparisonFigure = sns.FacetGrid(evaluationResults[~evaluationResults['somePredictionRefused']], 
                                            col = 'pruning',
                                            sharey = False, 
                                            height=5, 
                                            aspect=1).set(title = "Performance")
baseNetworkComparisonFigure.map(sns.violinplot, "spearman", "network", 
                                palette=["r", "b", "k", "y", "g"]
                               ).add_legend()
baseNetworkComparisonFigure.set(ylabel="Spearman correlation\nminus average over all methods")
plt.show()

In [ ]:
summary = evaluationResults[~evaluationResults['somePredictionRefused']]
summary = summary.groupby(["pruning", "network"]).mean()[["spearman"]].reset_index(["pruning", "network"])
summary = summary.merge(network_sizes)
summary.sort_values(['pruning', 'network'], inplace=True)
summary.to_csv("../results/" + VERSION + "/networksExperimentEvaluationSummary.csv")
print(summary)
baseNetworkComparisonFigureCompact = sns.scatterplot(data=summary[[p!="harsh" for p in summary["pruning"]]],
                x='numEdges',
                y='spearman', 
                hue='network')
baseNetworkComparisonFigureCompact.set_xscale("log")
baseNetworkComparisonFigureCompact.set(title="Density vs performance")
baseNetworkComparisonFigureCompact.legend(loc='center left', bbox_to_anchor=(1, 0.5))
